In [29]:
import os
import ROOT
#from ROOT import TF1, TCanvas, TMath, TH1C

ImportError: No module named ROOT

***
# Introductions to Research:

First of all, I recommend looking [here](https://lhcb.github.io/analysis-essentials/shell/) which includes a walk through of basics of python script(unfortunately no c++), **but** I think more importantly through shell navigation and use.  Competent shell navigation will allow you to more confidently create, access, organize, and share files..  all of which make this work much easier!
***


***
Put some fancy markdown here maybe some the decay channel of Higgs we are looking at with a Feynman diagram...
***

No idea what this $\uparrow$ is referring to (:  


***
# TODO: Load the data here.


In [16]:
import urllib.request as rq

In [21]:
#I've asked, but I don't know what data you'd like in here.. anyways here you go:  (take out the loop if you don't need it)
#does everyone have access to the hep server?  or are we planning on sharing data via git? 
url_names = ["http://opendata.web.cern.ch/record/5501/files/DoubleMu11.root",
"http://opendata.web.cern.ch/record/5501/files/DoubleE11.root",
"http://opendata.web.cern.ch/record/5501/files/DoubleMu12.root",   
"http://opendata.web.cern.ch/record/5501/files/DoubleE12.root",    
"http://opendata.web.cern.ch/record/5501/files/HZZ11.root",        
"http://opendata.web.cern.ch/record/5501/files/HZZ12.root",        
"http://opendata.web.cern.ch/record/5501/files/ZZ4mu11.root",
"http://opendata.web.cern.ch/record/5501/files/ZZ4mu12.root",
"http://opendata.web.cern.ch/record/5501/files/ZZ2mu2e11.root",
"http://opendata.web.cern.ch/record/5501/files/ZZ2mu2e12.root",
"http://opendata.web.cern.ch/record/5501/files/ZZ4e11.root",
"http://opendata.web.cern.ch/record/5501/files/ZZ4e12.root",
"http://opendata.web.cern.ch/record/5501/files/DY1011.root",     
"http://opendata.web.cern.ch/record/5501/files/DY1012.root",       
"http://opendata.web.cern.ch/record/5501/files/DY50TuneZ11.root",  
"http://opendata.web.cern.ch/record/5501/files/DY50TuneZ12.root",  
"http://opendata.web.cern.ch/record/5501/files/TTBar11.root",  
"http://opendata.web.cern.ch/record/5501/files/TTBar12.root"]

file_names = ["DoubleMu11.root", "DoubleE11.root", "DoubleMu12.root", "DoubleE12.root", "HZZ11.root", 
              "HZZ12.root", "ZZ4mu11.root", "ZZ4mu12.root", "ZZ2mu2e11.root", "ZZ2mu2e12.root",
             "ZZ4e11.root", "ZZ4e12.root", "DY1011.root", "DY1012.root", "DY50TuneZ11.root",
             "DY50TuneZ12.root", "TTBar11.root", "TTBar12.root"]

for i in range(len(url_names)):
    urlreq = rq.urlretrieve(url_names[i], file_names[i])

    #and now the files are wherever you're running this!
    


Running the code above saves all of the root files inside wherever you are running this notebook!  Can't tell you much about the meat of it (urllib.request), since I just found that from searching for a good python tool to use.

In [28]:

#creating file to transform into a data structure:

file_doubleMu11 = TFile('DoubleMu11.root', 'READ' )

#pull out whatever variables you want into whatever data structure you'd like?  
    #as for data structure (again, python really isn't a language I'm comfortable in) 
    #  I'd assume you'd want to make a panda dataset
    #that would be the case if you have more than one variable you want to read, compare, and keep organized per event

file_doubleMu11.Close()


NameError: name 'TFile' is not defined

***
A function to determine the decaying exponential fit to the background and the gaussian fit to the signal. A gaussian integration of the background and background+signal in the signal region is performed to determine the amount of signal and the signal-to-noise ratio (SNR).

Documentation for the [TF1 class](https://root.cern.ch/doc/master/classTF1.html), at the Root Reference Guide.  
Do not be afraid to search through the Reference Guide! If you don't underestand what parameters are going into a function, simply look up the class that function is being called on.  
  
  **For Example:**  
The below definition of `falling_exponential_fit`, you can see it's being defined with a TF1 classification. Go to the Root Reference Guide and search the [TF1 class](https://root.cern.ch/doc/master/classTF1.html). There are examples listed, but scroll below to the section labelled **Public Member Functions**. The first function used in the code is a Constructor, which is constructing our 1-Dimension fuction.  Looking at the constructor used we see it's using a name, a formula, then two numbers. That corresponds with the constructor on the page marked as
     
     `TF1 (const char *name, const char *formula, Double_t xmin, Double_t xmax, Option_t *option)`,  
where the `option` was not used. In order to find out what the formula input needs to be, there's a helpful [link for the TFormula class](https://root.cern.ch/doc/master/classTFormula.html) right in the description of the function.  
  
Next, to find the formulas `SetParameters` and `SetParLimits`, simply search the TF1 class page for the function name and the corresponding input to find all the information you need.

**~~this is not a good example~~**  
  
  
Continue in this fashion whenever you're confused by a section of code you see. It will be immensly helpful while learning how to use the ROOT framework.  Once you get comfortable using the Root Reference Guide, it'll become instrumental in creating your code.  ~~It's a bit easier to understand if you're using c++ to code, as all of the examples are likely in c++, as well as most of the documentation..  If you're using python, then I'm assuming you may just ignore any asterisks(\*) or ampersands(&), or most unfamiliar things (disregard the pointer system)~~

In [30]:

# @par canvas_name The name of the canvas to display the analysis of the histogram @hist.
# @par hist The histogram that is being analyzed.
def analyze_histogram(canvas_name, hist):
    # Construct the falling exponential fit to be used for the background
    falling_exponential_fit = ROOT.TF1("fallingExponentialFit", "[0]*exp(-1*[1]*x)", 0, 75)
    falling_exponential_fit.SetParameters(100, 0.01)
    falling_exponential_fit.SetParLimits(0, 0.01, 10000)
    falling_exponential_fit.SetParLimits(1, 0.00001, 100)
    
    # Construct the gaussian fit to be used to fit the signal region
    gaussian_fit = ROOT.TF1("gaussianFit", "[0]*exp(-0.5*((x-[1])/[2])^2)", 75, 110)
    gaussian_fit.SetParameters(100, 100, 20)
    gaussian_fit.SetParLimits(0, 0, 1000)
    gaussian_fit.SetParLimits(1, 50, 150)
    gaussian_fit.SetParLimits(2, 0.1, 100)
    gaussian_fit.SetLineColor(ROOT.kGreen)
    
    # Create the canvas
    canvas = ROOT.TCanvas(canvas_name, canvas_name, 700, 700)
    hist.Fit(falling_exponential_fit, "LR+")
    hist.Fit(gaussian_fit, "LR+")
    
    # Perform the integration and calculate SNR
    wrapped_fall_exp = ROOT.Math.WrappedTF1(falling_exponential_fit)
    wrapped_gauss = ROOT.Math.WrappedTF1(gaussian_fit)
    integrator = ROOT.Math.GaussIntegrator()
    integrator.SetFunction(wrapped_fall_exp)
    integrator.SetRelTolerance(0.001)
    noise_integration = integrator.Integral(75, 125)
    integrator.SetFunction(wrapped_gauss)
    signal_integration = integrator.Integral(75, 125)
    _signal = signal_integration - noise_integration
    snr = _signal/noise_integration
    
    # Print the output and draw onto canvas.
    print("The measured noise within the signal region was: " + str(noise_integration) + "\n")
    print("The measured signal within the signal region was: " + str(_signal) + "\n")
    print("The signal to noise ratio (SNR) is: " + str(snr) + "\n")
    print("=====\n")
    print('The mass of the Z boson is ' + str(gaussian_fit.GetParameter(1)) + ' +- ' + str(gaussian_fit.GetParError(1)) + ' GeV/c^2\n')
    hist.Draw()
    png_filename = canvas_name + ".png"
    canvas.SaveAs(png_filename)

I'm guessing there'll be some sort of description here on what you're trying to do?


In [20]:
# Generating output
numBins = 100
lowerXRange = 0
upperXRange = 260

# Create the histograms
zBosonHist = ROOT.TH1C("zBosonHist", "Histogram of Invariant Masses of Dilepton Decay", numBins,
                       lowerXRange, upperXRange)
zBosonElectronHist = ROOT.TH1C("zBosonElectronHist", "Histogram of Invariant Masses of Dielectron Decays", 
                               numBins, lowerXRange, upperXRange)
zBosonMuonHist = ROOT.TH1C("zBosonMuonHist", "Histogram of Invariant Masses of Dimuon Decays", 
                           numBins, lowerXRange, upperXRange)

# Fill the data
#  for(it = mass_map.begin(); it != mass_map.end(); it++) {
#    vector<double> masses = it->second;
#    for(unsigned int i = 0; i < masses.size(); ++i) {
#      zBosonHist->Fill(masses[i])
#      if(it->first.compare("e"))
#          zBosonElectronHist->Fill(masses[i]);
#      else if(it->first.compare("m"))
#        zBosonMuonHist->Fill(masses[i]);
#    }}
# Analyze the histograms

print("=================================================\n")
print("Analyzing all of the dilepton decays...\n")
analyze_histogram("overallHistCanvas", zBosonHist)
print("=================================================\n")
print("Analyzing just the dielectron decays\n")
analyze_histogram("dielectronHistCanvas", zBosonElectronHist)
print("=================================================\n")
print("Analyzing just the dimuon decays\n")
analyze_histogram("dimuonHistCanvas", zBosonMuonHist)
print("=================================================\n")

SyntaxError: invalid syntax (<ipython-input-20-4da72aa7697f>, line 15)

![title](overallHistCanvas.png)

Investigating lepton universality...
![title](dielectronHistCanvas.png)
![title](dimuonHistCanvas.png)